In [ ]:
import os
import re
import glob
import pickle

data_dir = "/root/autodl-tmp/embeddings/mmlu/qwen3/all_results"
all_data_subject = ['high_school_physics', 'high_school_chemistry', 'high_school_biology', 'college_mathematics']
# get all pkl files under the deepseek_data_dir
# pkl_files = glob.glob(os.path.join(data_dir, "*.pkl"))
# print(f"Found {len(pkl_files)} pkl files.")
pkl_files = [
    f"{data_dir}/{subject_name}_results.pkl" for subject_name in all_data_subject
]
for pkl_file in pkl_files:
    print(pkl_file)

all_data = []

for subject_name, pkl_file in zip(all_data_subject, pkl_files):
    # match = re.search(r"\{(.*?)\}", pkl_file)
    # subject_name = match.group(1)
    # match = re.search(r"\{(.*?)\}", pkl_file)
    print(pkl_file)
    with open(pkl_file, "rb") as f:
        data = pickle.load(f)
        all_data.extend([data])
        all_data_subject.append(subject_name)
print(f"Loaded {len(all_data)} items from pkl files.")
print(type(data['test']))
print(all_data_subject)

/root/autodl-tmp/embeddings/mmlu/qwen3/all_results/high_school_physics_results.pkl
/root/autodl-tmp/embeddings/mmlu/qwen3/all_results/high_school_chemistry_results.pkl
/root/autodl-tmp/embeddings/mmlu/qwen3/all_results/high_school_biology_results.pkl
/root/autodl-tmp/embeddings/mmlu/qwen3/all_results/college_mathematics_results.pkl
/root/autodl-tmp/embeddings/mmlu/qwen3/all_results/high_school_physics_results.pkl
{'test': [{'question': 'The plates of a capacitor are charged to a potential difference of 5 V. If the capacitance is 2 mF, what is the charge on the positive plate?', 'answer': 1, 'embedding': {'layer_36': tensor([[ 0.7064,  0.6042, -0.7202,  ...,  0.2823,  0.7714,  0.2524],
        [-0.0819, -0.7854,  0.2963,  ...,  0.5831, -0.3179,  0.3604],
        [-0.6991,  0.0623, -0.1778,  ..., -0.1120, -0.5438,  0.2498],
        [ 0.0746,  0.1189,  0.6017,  ..., -0.7535,  0.0904, -0.8626]],
       device='cuda:0')}}, {'question': 'Which of these quantities decreases as the inverse squ

In [64]:
import torch

# group all test data into a single list
all_test_data = []
all_dev_data = []
for i, data in enumerate(all_data):
    test_data = data.get("test", [])
    dev_data = data.get("dev", [])
    for item in test_data:
        item['subject'] = all_data_subject[i]
    for item in dev_data:
        item['subject'] = all_data_subject[i]
    all_test_data.extend(test_data)
    all_dev_data.extend(dev_data)

print(f"Total test data: {len(all_test_data)}")
print(f"Total dev data: {len(all_dev_data)}")
print(all_test_data[0])

Total test data: 764
Total dev data: 20
{'question': 'The plates of a capacitor are charged to a potential difference of 5 V. If the capacitance is 2 mF, what is the charge on the positive plate?', 'answer': 1, 'embedding': {'layer_36': tensor([[ 0.7064,  0.6042, -0.7202,  ...,  0.2823,  0.7714,  0.2524],
        [-0.0819, -0.7854,  0.2963,  ...,  0.5831, -0.3179,  0.3604],
        [-0.6991,  0.0623, -0.1778,  ..., -0.1120, -0.5438,  0.2498],
        [ 0.0746,  0.1189,  0.6017,  ..., -0.7535,  0.0904, -0.8626]],
       device='cuda:0')}, 'subject': 'high_school_physics'}


In [65]:
high_school_physics_dev_data = [
    item for item in all_dev_data if item['subject'] == 'high_school_physics'
]
high_school_chemistry_dev_data = [
    item for item in all_dev_data if item['subject'] == 'high_school_chemistry'
]

In [ ]:
import random
from tqdm import tqdm
from typing import List

LARYER_NAME = 'layer_36'  # 'layer_32' or 'layer_36'

def get_xi_all_y_embeddings(data_item):
    """
    Extract all embeddings from the provided data.
    """
    return data_item['embedding'][LARYER_NAME]

def get_Xi_matrix(x_item_tensor: torch.Tensor):
    """
    Extract the Xi matrix from the provided data.
    """
    # Xi_matrix shape: (dim_feat, dim_feat)
    # Xi(x) = \sum_y \xi(x,y) \xi(x,y)^T
    Xi = torch.einsum('ij,ik->jk', x_item_tensor, x_item_tensor)

    return Xi

def get_mean_xi_yi_tensor(x_multi_tensor: torch.Tensor, answer: List[int]):
    # x_multi_tensor shape: (num_samples, num_y, dim_feat)
    # answer: List of answers for each sample
    xi_yi_tensor = torch.stack([
        x_multi_tensor[i, answer[i]] for i in range(x_multi_tensor.shape[0])
    ])
    mean_xi_yi_tensor = torch.mean(xi_yi_tensor, dim=0)
    return mean_xi_yi_tensor

def get_mean_Xi_matrix(x_multi_tensor: torch.Tensor):
    """
    Calculate the mean Xi matrix from multiple tensors.
    """
    # x_multi_tensor shape: (num_samples, num_y, dim_feat)
    # Xi(x) = \frac{1}{n} \sum_{i}^n \sum_y \xi(x_i,y) \xi(x_i,y)^T
    Xi = torch.einsum('ijk,ijl->ikl', x_multi_tensor, x_multi_tensor)
    mean_Xi = torch.mean(Xi, dim=0)

    return mean_Xi

def get_random_samples(dev_data, num_samples=5):
    if len(dev_data) < num_samples:
        raise ValueError("Not enough examples in dev data to pick from.")
    sampled_data =  random.sample(dev_data, num_samples)
    random_picked_dev_tensor = torch.stack(
        [item['embedding'][LARYER_NAME] for item in sampled_data]
    )
    
    all_xi_yi_list = []
    for data_item in sampled_data:
        answer: int = int(data_item['answer'])
        xi_yi_tensor = data_item['embedding'][LARYER_NAME][answer]
        all_xi_yi_list.append(xi_yi_tensor)
    
    all_xi_yi_tensor = torch.stack(all_xi_yi_list)  # shape: (num_samples, dim_feat)

    return all_xi_yi_tensor, random_picked_dev_tensor

def get_lambda_1_Xq_Xn(Xq_matrix, Xn_pinv):
    """
    Calculate the lambda_1 of matrix using the provided formula.
    """
    combined_matrix = Xq_matrix @ Xn_pinv
    lambda_1 = torch.linalg.eigvalsh(combined_matrix).max().item()
    return lambda_1


In [67]:
def get_metrics(data_list):
    data_xi = torch.stack(
        [get_xi_all_y_embeddings(item) for item in data_list], dim=0
    )
    mean_xi_yi_tensor = get_mean_xi_yi_tensor(data_xi, [item['answer'] for item in data_list])
    mean_Xi = get_mean_Xi_matrix(data_xi)
    mean_Xi_pinv = torch.linalg.pinv(mean_Xi)

    return mean_xi_yi_tensor, mean_Xi, mean_Xi_pinv

In [68]:
(
    high_school_physics_dev_data_mean_xi_yi, 
    high_school_physics_dev_data_mean_Xi,
    high_school_physics_dev_data_mean_Xi_pinv
) = get_metrics(high_school_physics_dev_data)

print(high_school_physics_dev_data_mean_xi_yi.shape)
print(high_school_physics_dev_data_mean_Xi.shape)
print(high_school_physics_dev_data_mean_Xi_pinv.shape)

KeyError: 'layer_32'

In [37]:
(
    high_school_chemistry_dev_data_mean_xi_yi, 
    high_school_chemistry_dev_data_mean_Xi,
    high_school_chemistry_dev_data_mean_Xi_pinv
) = get_metrics(high_school_chemistry_dev_data)

print(high_school_chemistry_dev_data_mean_xi_yi.shape)
print(high_school_chemistry_dev_data_mean_Xi.shape)
print(high_school_chemistry_dev_data_mean_Xi_pinv.shape)

torch.Size([4096])
torch.Size([4096, 4096])
torch.Size([4096, 4096])


In [38]:
from tqdm import tqdm

high_school_physics_test_data = [
    item for item in all_test_data if item['subject'] == 'high_school_physics'
]

lambda_1_high_school_physics_all = []
lambda_1_high_school_chemistry_all = []
for i, data in enumerate(tqdm(high_school_physics_test_data)): 
    test_embeddings = get_xi_all_y_embeddings(data)
    Xq_matrix = get_Xi_matrix(test_embeddings)
    lambda_1_high_school_physics = get_lambda_1_Xq_Xn(Xq_matrix, high_school_physics_dev_data_mean_Xi_pinv)
    lambda_1_high_school_physics_all.append(lambda_1_high_school_physics)

    lambda_1_high_school_chemistry = get_lambda_1_Xq_Xn(Xq_matrix, high_school_chemistry_dev_data_mean_Xi_pinv)
    lambda_1_high_school_chemistry_all.append(lambda_1_high_school_chemistry)

100%|██████████| 151/151 [02:35<00:00,  1.03s/it]


In [39]:
print(sum(lambda_1_high_school_physics_all))
print(sum(lambda_1_high_school_chemistry_all))

313.8731565475464
281.8049590587616


In [41]:
high_school_physics_correct = []
high_school_chemistry_correct = []

alpha_high_school_physics = high_school_physics_dev_data_mean_Xi_pinv @ high_school_physics_dev_data_mean_xi_yi
alpha_high_school_chemistry = high_school_chemistry_dev_data_mean_Xi_pinv @ high_school_chemistry_dev_data_mean_xi_yi

for i, data in enumerate(tqdm(high_school_physics_test_data)): 
    test_embeddings = get_xi_all_y_embeddings(data)
    hat_P_high_school_physics = alpha_high_school_physics @ test_embeddings.T
    argmax_hat_P_high_school_physics = torch.argmax(hat_P_high_school_physics).item()

    hat_P_high_school_chemistry = alpha_high_school_chemistry @ test_embeddings.T
    argmax_hat_P_high_school_chemistry = torch.argmax(hat_P_high_school_chemistry).item()

    answer: int = int(data['answer'])

    high_school_physics_correct.append(int(argmax_hat_P_high_school_physics == answer))
    high_school_chemistry_correct.append(int(argmax_hat_P_high_school_chemistry == answer))

100%|██████████| 151/151 [00:00<00:00, 539.74it/s]


In [ ]:
print(sum(high_school_physics_correct)/len(high_school_physics_correct))
print(sum(high_school_chemistry_correct)/len(high_school_chemistry_correct))

0.5033112582781457
0.46357615894039733


In [47]:
# get the lambda_1 of high school physics correct answers
lambda_1_high_school_physics_correct = []
lambda_1_high_school_physics_incorrect = []
for correct_result in high_school_physics_correct:
    if correct_result == 1:
        lambda_1_high_school_physics_correct.append(
            lambda_1_high_school_physics_all[correct_result]
        )
    else:
        lambda_1_high_school_physics_incorrect.append(
            lambda_1_high_school_physics_all[correct_result]
        )

print(sum(lambda_1_high_school_physics_correct))
print(sum(lambda_1_high_school_physics_incorrect))

138.3744993209839
196.94532752037048


In [48]:
quadrant_count = {
    "physics_correct_chemistry_correct": [],
    "physics_correct_chemistry_wrong": [],
    "physics_wrong_chemistry_correct": [],
    "physics_wrong_chemistry_wrong": [],
}

for i, (p, c) in enumerate(zip(high_school_physics_correct, high_school_chemistry_correct)):
    if p == 1 and c == 1:
        key = "physics_correct_chemistry_correct"
    elif p == 1 and c == 0:
        key = "physics_correct_chemistry_wrong"
    elif p == 0 and c == 1:
        key = "physics_wrong_chemistry_correct"
    else:
        key = "physics_wrong_chemistry_wrong"
    
    quadrant_count[key].append(
        {
            "lambda_1_physics": lambda_1_high_school_physics_all[i],
            "lambda_1_chemistry": lambda_1_high_school_chemistry_all[i]
        }
    )

In [51]:
print(len(quadrant_count['physics_correct_chemistry_correct']))
print(len(quadrant_count['physics_correct_chemistry_wrong']))
print(len(quadrant_count['physics_wrong_chemistry_correct']))
print(len(quadrant_count['physics_wrong_chemistry_wrong']))

38
38
32
43


In [56]:
import numpy as np

lambda_1_physics_correct_chemistry_correct = [
    item['lambda_1_physics'] for item in quadrant_count['physics_correct_chemistry_correct']
]
print(sum(lambda_1_physics_correct_chemistry_correct) / len(lambda_1_physics_correct_chemistry_correct))

for key in quadrant_count.keys():
    lambda_1_physics = [item['lambda_1_physics'] for item in quadrant_count[key]]
    lambda_1_chemistry = [item['lambda_1_chemistry'] for item in quadrant_count[key]]
    print(f"{key}:")
    print(f"  lambda_1_physics: {sum(lambda_1_physics) / len(lambda_1_physics)} with std: {np.std(lambda_1_physics)}")
    print(f"  lambda_1_chemistry: {sum(lambda_1_chemistry) / len(lambda_1_chemistry)} with std: {np.std(lambda_1_chemistry)}")

2.0777312328940942
physics_correct_chemistry_correct:
  lambda_1_physics: 2.0777312328940942 with std: 0.20883153909288826
  lambda_1_chemistry: 1.790143543168118 with std: 0.24247625560093186
physics_correct_chemistry_wrong:
  lambda_1_physics: 1.9358964091853093 with std: 0.3369237241656967
  lambda_1_chemistry: 1.8400833512607373 with std: 0.3239041872156546
physics_wrong_chemistry_correct:
  lambda_1_physics: 2.058322060853243 with std: 0.2946749583086163
  lambda_1_chemistry: 1.9335137493908405 with std: 0.25618476182799743
physics_wrong_chemistry_wrong:
  lambda_1_physics: 2.2206744232843088 with std: 0.38291611803581355
  lambda_1_chemistry: 1.9066022579060045 with std: 0.3564851259960243


In [4]:
all_test_Xi_matrix = torch.zeros((4096, 4096), device='cuda:0')
for i, data in enumerate(tqdm(all_test_data)):
    test_embeddings = get_xi_all_y_embeddings(data)
    Xi_matrix = get_Xi_matrix(test_embeddings)  # shape: (dim_feat, dim_feat)
    all_test_Xi_matrix = all_test_Xi_matrix * i / (i+1) + Xi_matrix / (i+1)

all_dev_Xi_matrix = torch.zeros((4096, 4096), device='cuda:0')
for i, data in enumerate(tqdm(all_dev_data)):
    dev_embeddings = get_xi_all_y_embeddings(data)
    Xi_matrix = get_Xi_matrix(dev_embeddings)  # shape: (dim_feat, dim_feat)
    all_dev_Xi_matrix = all_dev_Xi_matrix * i / (i+1) + Xi_matrix / (i+1)

test_data_num = len(all_test_data)
dev_data_num = len(all_dev_data)
all_data_num = test_data_num + dev_data_num
all_Xi_matrix = all_test_Xi_matrix * test_data_num / all_data_num + all_dev_Xi_matrix * dev_data_num / all_data_num

 31%|███       | 236/764 [00:00<00:00, 2358.32it/s]

100%|██████████| 20/20 [00:00<00:00, 1511.54it/s]


In [6]:
mean_test_xi_yi_tensor = torch.zeros((4096,), device='cuda:0')
for i, data in enumerate(tqdm(all_test_data)):
    test_embeddings = get_xi_all_y_embeddings(data)
    answer: int = int(data['answer'])
    xi_yi_tensor = test_embeddings[answer]
    mean_test_xi_yi_tensor = mean_test_xi_yi_tensor * i / (i+1) + xi_yi_tensor / (i+1)

mean_dev_xi_yi_tensor = torch.zeros((4096,), device='cuda:0')
for i, data in enumerate(tqdm(all_dev_data)):
    dev_embeddings = get_xi_all_y_embeddings(data)
    answer: int = int(data['answer'])
    xi_yi_tensor = dev_embeddings[answer]
    mean_dev_xi_yi_tensor = mean_dev_xi_yi_tensor * i / (i+1) + xi_yi_tensor / (i+1)

mean_xi_yi_tensor = mean_test_xi_yi_tensor * test_data_num / all_data_num + mean_dev_xi_yi_tensor * dev_data_num / all_data_num

100%|██████████| 20/20 [00:00<00:00, 15899.56it/s]


In [13]:
all_Xi_matrix_pinv = torch.linalg.pinv(all_Xi_matrix)
alpha_all = torch.matmul(all_Xi_matrix_pinv, mean_xi_yi_tensor)
print(alpha_all, alpha_all.shape)

tensor([ 0.0117,  0.0135,  0.0124,  ...,  0.0066, -0.0078,  0.0046],
       device='cuda:0') torch.Size([4096])


In [12]:
alpha_all = torch.linalg.lstsq(all_Xi_matrix, mean_xi_yi_tensor).solution
print(alpha_all, alpha_all.shape)

tensor([-1.2024,  0.6085, -1.0478,  ...,  0.7426,  0.0662,  0.2007],
       device='cuda:0') torch.Size([4096])


In [22]:
# Get the R(x_Q, y)
R_all = []
choose_answer = []
for i, data in enumerate(tqdm(all_test_data)):
    answer: int = int(data['answer'])
    true_P = torch.zeros((4,), device='cuda:0')
    embedding = get_xi_all_y_embeddings(data)
    hat_P = alpha_all @ embedding.T
    choose_answer.append(torch.argmax(hat_P).item())
    R = true_P - hat_P
    R_all.append(R)

100%|██████████| 764/764 [00:00<00:00, 6717.75it/s]


In [18]:
R_all

[tensor([ 0.1799, -0.1226,  0.2006, -0.2579], device='cuda:0'),
 tensor([-0.5202,  0.3238,  0.0097,  0.1867], device='cuda:0'),
 tensor([ 0.2825,  0.2790,  0.4196, -0.9812], device='cuda:0'),
 tensor([ 0.0319, -0.0955, -0.2247,  0.2883], device='cuda:0'),
 tensor([ 0.5216,  0.3883,  0.0599, -0.9697], device='cuda:0'),
 tensor([ 0.3985,  0.1643, -0.8573,  0.2946], device='cuda:0'),
 tensor([-0.8460,  0.1272,  0.4486,  0.2702], device='cuda:0'),
 tensor([ 0.0714,  0.0047, -0.6998,  0.6237], device='cuda:0'),
 tensor([-0.5497,  0.0915,  0.3901,  0.0680], device='cuda:0'),
 tensor([ 0.2204, -0.1925, -0.1803,  0.1524], device='cuda:0'),
 tensor([ 0.4775,  0.2057, -0.1921, -0.4911], device='cuda:0'),
 tensor([ 0.0190,  0.3410,  0.1923, -0.5522], device='cuda:0'),
 tensor([ 0.4155, -0.0596,  0.4618, -0.8177], device='cuda:0'),
 tensor([ 0.3082, -0.1979, -0.0973, -0.0130], device='cuda:0'),
 tensor([ 0.2823, -0.3839, -0.2256,  0.3271], device='cuda:0'),
 tensor([-0.7822,  0.2592,  0.1572,  0.3

In [28]:
correct = 0
for i, data in enumerate(tqdm(all_test_data)):
    answer: int = int(data['answer'])
    choose_answer_data = choose_answer[i]
    if choose_answer_data == answer:
        correct += 1
    else:
        print()

print(f"Correct predictions: {correct} out of {len(all_test_data)}, accuracy: {correct / len(all_test_data):.4f}")

100%|██████████| 764/764 [00:00<00:00, 721397.63it/s]




















Correct predictions: 745 out of 764, accuracy: 0.9751


In [14]:
num_example = 5
results = []
correct = 0
with tqdm(all_test_data) as pbar:
    for i, data in enumerate(pbar):
        xi_all_y = get_xi_all_y_embeddings(data) # shape: (num_y, dim_feat)

        # randomly pick 5 examples from dev_data
        all_xi_yi_tensor, random_picked_dev_tensor = get_random_samples(all_dev_data, num_example) # shape: (num_example, num_y, dim_feat)
        mean_Xi = get_mean_Xi_matrix(random_picked_dev_tensor) # shape: (dim_feat, dim_feat)
        mean_xi_yi_tensor = torch.mean(all_xi_yi_tensor, dim=0)  # shape: (dim_feat)
        mean_xi_yi_tensor = mean_xi_yi_tensor.unsqueeze(-1)  # shape: (dim_feat, 1)

        # solve for results
        # alpha = torch.linalg.pinv(mean_Xi) @ mean_xi_yi_tensor # shape: (dim_feat, 1)
        alpha = torch.linalg.lstsq(mean_Xi, mean_xi_yi_tensor).solution  # shape: (dim_feat, 1)
        hat_P = xi_all_y @ alpha # shape: (num_y, 1)
        one_hot_hat_P = torch.argmax(hat_P, dim=0)  # shape: (1,)

        # get metrics
        ## correct
        answer = int(data['answer'])
        is_correct = one_hot_hat_P == answer
        correct += int(is_correct)
        acc = (correct / (i + 1)) * 100
        pbar.set_postfix(acc=f"{acc:.4f}")

        ## Xi rank
        xi_rank = torch.linalg.matrix_rank(mean_Xi)  # shape: (1,)
        ## loss
        one_hot_true_P = torch.zeros_like(hat_P)
        one_hot_true_P[answer] = 1.0  # shape: (num
        l2_loss = torch.norm(hat_P - one_hot_true_P, p=2)  # shape: (1,)

        results.append({
            "question": data['question'],
            "answer": answer,
            "predicted_answer": int(one_hot_hat_P),
            "is_correct": is_correct.item(),
            "hat_P": hat_P.squeeze().tolist(),  
            "rank": xi_rank.item(),
            "loss": l2_loss.item()
        })
    

100%|██████████| 764/764 [55:05<00:00,  4.33s/it, acc=26.3089] 


In [2]:
import pickle
import torch

file_name = [
    "/root/autodl-tmp/embeddings/mmlu/qwen3/all_results/high_school_biology_results.pkl",
    "/root/autodl-tmp/embeddings/mmlu/qwen3/all_results/high_school_chemistry_results.pkl",
    "/root/autodl-tmp/embeddings/mmlu/qwen3/all_results/high_school_physics_results.pkl",
    "/root/autodl-tmp/embeddings/mmlu/qwen3/all_results/college_mathematics_results.pkl",
]

data_all = []

for file in file_name:
    with open(file, "rb") as f:
        data = pickle.load(f)
        data_all.extend(data['test'])

In [10]:
print(data_all[0])

{'question': 'In a population of giraffes, an environmental change occurs that favors individuals that are tallest. As a result, more of the taller individuals are able to obtain nutrients and survive to pass along their genetic information. This is an example of', 'answer': 0, 'embedding': {'layer_36': tensor([[ 0.6825,  0.6348, -0.7862,  ..., -0.4176,  0.8304,  0.5535],
        [-0.0748, -0.7661, -0.0061,  ...,  0.8519, -0.4966,  0.3980],
        [ 0.1108,  0.0922,  0.2117,  ..., -0.2703, -0.2304, -0.2725],
        [-0.7185,  0.0391,  0.5806,  ..., -0.1640, -0.1035, -0.6790]],
       device='cuda:0')}}


In [4]:
import json

with open("qwen_question.data.json", "r") as f:
    qwen_data = json.load(f)

qwen_data_all = qwen_data['results']

In [11]:
for item in qwen_data_all:
    question_full = item['question']
    first_line = question_full.split('\n', 1)[0]
    first_line = first_line.split(' ', 1)[1]
    for data in data_all:
        if first_line in data['question']:
            item['embedding'] = data['embedding']

{'question': 'Question: In a population of giraffes, an environmental change occurs that favors individuals that are tallest. As a result, more of the taller individuals are able to obtain nutrients and survive to pass along their genetic information. This is an example of\nA. directional selection.\nB. stabilizing selection.\nC. sexual selection.\nD. disruptive selection.',
 'true_answer': 'A',
 'pred_answer': 'A',
 'response': 'none',
 'is_correct': True,
 'few_shot_prompt': 'Question: Which of the following is not a way to form recombinant DNA?\nA. Translation\nB. Conjugation\nC. Specialized transduction\nD. Transformation\nAnswer: A\n\nQuestion: A mutation in a bacterial enzyme changed a previously polar amino acid into a nonpolar amino acid. This amino acid was located at a site distant from the enzyme’s active site. How might this mutation alter the enzyme’s substrate specificity?\nA. By changing the enzyme’s pH optimum\nB. By changing the enzyme’s location in the cell\nC. By cha